## Atoms

AtomBuilder3 (AB3) builds charge field atomic models selected by 
the user. AB3 is a copy of mBuilder, utilizing a datafile, and one 
and two directional proton configuration slot groups replacing earlier 
versions, AtomBuilder (AB) and AtomBuilder2 (AB2). AB and AB2 are now 
in the additional notebooks folder as history files. mbuilder carries 
on as the day-to-day working program intended to develop a 3D charge 
field model rendering of molecular matter selected by the user.  

Operating instructions. Start by clicking on the "Atom data" cell,
about 10 cells below. Next, from the toolbar's Cell dropdown selection 
menu choose "Run all Above". When the gui in the cell above "Atom data" 
becomes visible, make any changes to it or select an atom from the 
Periodic table. Next, with the gui or the Periodic table cell still
active, select "Run all Below" from the Cell dropdown menu. You'll then 
view the main program outputs from the tab enclosure cell near the 
bottom. For any changes, repeat - go back to the gui or Periodic 
table, make changes, then select "Run all Below". These instructions 
are repeated in the "Atom data" cell.

Atoms is intended to introduce Charge field theory - see the 
notebook's end cell's source references.                                          

In [ ]:
from pythreejs import *
from IPython.display import display
from ipywidgets import HBox, VBox, Button, ButtonStyle, GridspecLayout
import ipywidgets as widgets
from math import sqrt, tan, copysign
from random import random , randint , seed

In [ ]:
# constants
PHI = (1+sqrt(5))/2  # calculating phi-based-pi, PI_PHI.
PI_PHI = 4/sqrt(PHI)
PROTON_RADIUS = 0.05 # proton radius, used by all atomic distances
CAROUSEL_ROTATION_SECONDS = 5.0 # Problem Here # time in secs for the carousel to make one rotation

In [ ]:
# For pythreejs display, sizing the view, camera and adding lights.
VIEW_WIDTH = 600
VIEW_HEIGHT = 400
# Horizontal z axis.
camera_z_axis_horiz = CombinedCamera(position=[0, 0, 10], up=[0, 1, 0], width=VIEW_WIDTH, height=VIEW_HEIGHT)
# Vertical z axis, 
camera_z_axis_vert = CombinedCamera(position=[0, 0, 10], up=[0, 0, 1], width=VIEW_WIDTH, height=VIEW_HEIGHT)
# camera will be selected in the render cell since it is one of the user controlled variables.

# See renderer cell for info, non-jason compliant pythreejs orbital camera user warning 
key_light = Directional_light(position=[0, 5, 5])
ambient_light = AmbientLight()

In [ ]:
import pandas as pd
import numpy as np

# Sizing the users' local display of tabular data if need_ed.
#pd.set_option('max_columns', 4, 'max_rows', 10, 'max_colwidth', 12)

# Don't read the slotLayout diags from Cr6-ElementsdataFrame,
# this programm creates its own SL diagram.
#filename = 'data/Cr6-Elements.csv'   
FILENAME = 'data/Cr6-Elements-M.csv' # changing slot configurations 
df_raw = pd.read_csv(FILENAME) 
#df_raw = df_raw.loc[:,:('CanBind')]
# Change. The SL diags from Cr6-Elements are read but not used
df_raw = df_raw.loc[:,:('SL4')] 

# Uncomment df_raw or df_raw.info() to view
# df_raw
# df_raw.info()

# Make changes to the dataFrame created by pd.read_csv(filename)'s 
# method defaults. The data type constrains how you may work with 
# the data.
df = df_raw.copy()
# Columns "SlotLayout" through "SL4" contain many non-value entries
# that pandas interprets as NaNs. Convert the NaNs to blank spaces.
df = df.fillna(" ")  

# Duplicating "AtomicNumber", one as a string,the other an int.
df["AtomicSlotGrp"] = df["AtomicNumber"].copy()
# df["AtomicNumber"] defaults to an integer. Make it string.
df["AtomicNumber"] = df["AtomicNumber"].astype(str)

# Make "CanBind" boolean.
df["CanBind"] = np.where(df["CanBind"] == "Y", True, False)

# Proton stack locations need to be float
df["PX"] = df["PX"].astype("float")
df["PY"] = df["PY"].astype("float")
df["PZ"] = df["PZ"].astype("float")
df["P2P3"] = df["P2P3"].astype("float")
df["P12"] = df["P12"].astype("float")
df["PE"] = df["PE"].astype("float")

# Uncomment df or df.info() to view
#df_raw
#df
#df.info()

In [ ]:
"""
Elements and Molecules 

'elements'is a list of tuples used by the program to  
identify or re-select a desired atom from the gui 
dropdown or periodic table. Also used as the optional
3D atom's atomic label. 'elements' is taken from the 
df['AtomicTuple'] column.  

Starting with 1710 rows. After drop_duplicates, the 
'AtomicTuple' row index increments by 19's, 0 to 1709, 
19x expanded. The Series.array method renumbers the 
index back to the desired 0-89. The two steps are 
chained.

Next, atomicTups has been read as a list of strings. 
Those strings are converted into tuples by using 
the map and eval methods.
"""
atomic_tups = df['AtomicTuple'].drop_duplicates().array
elements = list(map(eval, atomic_tups))
#elements

# Molecules. A user will select the 'molecule': 
# numberM (1-4) from this set.
molecules = {'H2':1,'C2':2,'N2':3,'O2':4}
#molecules

# Each triplet below (in numberN position) incl: 1. total num of  
# atoms; 2. the atomic number for the first; and 2. second atom.
mole_config = [(2,1,1), (2,6,6), (2,7,7), (2,8,8)]
#moleConfig

In [ ]:
# The periodic table's 90 buttons. Each button  
# includes: 1. Atomic type color, clist, worked 
# in steps first-fifth. 2. Atomic symbol, dlist. 
# 3. Layout width and layout height

# First, the ten Atomic types, tenATypes. 
# Note. tenATypes = list(df['Atomic ...duplicates())
# works ok, the explicit written list is simpler). 
ten_a_types = list(df['AtomicType'].drop_duplicates())  
#tenATypes = ['nonmetal','noble_gas','alkali_metal','alkaline_earth','metal_loid',
#             'halogen','basic_metal','transition_metal','lanthanide','actinide']
#tenATypes

# Second - The 10 atomic type colors, tenColors. 
# Lined up with tenATypes
ten_colors = ['skyblue','cyan','tomato','orange','springgreen','powderblue','palegreen',
         'yellow','thistle','violet']

# zip together the type/color dictionary
type_color_dict = dict(zip(ten_a_types, ten_colors))
#typeColorDict

# Fourth - creating a 1710 row number/type tuple
# list which is reduced to 90 unique rows.
df['al_lATypes'] = list(zip(df.AtomicNumber, df.AtomicType))
ATypes = df['al_lATypes'].drop_duplicates().array 
#ATypes

# Fifth - Build the list of 90 colors, clist,
# used to color the Periodic table's 90 buttons.
clist = []
for i in range(90):
    clist.append(type_color_dict[ATypes[i][1]])
#clist

# The atomic symbols for the Periodic table. 
dlist = df['AtomicSymbol'].drop_duplicates().array

# Creating the list of buttons
a_buttons = []
for i in range(90):
    # add the atomic symbols and colors to a button
    button = Button(description=dlist[i], style=dict(button_color=clist[i]))
    # add each button's border, layout width and height.
    button.layout = widgets.Layout(border = '1px solid black') 
    button.layout.width = '100%'
    button.layout.height = '100%'  #'40px'
    a_buttons.append(button)
    
# Layout the 90 buttons in periodic table form
grid = GridspecLayout(10, 18)

grid[0,0] = a_buttons[0]
grid[0,17] = a_buttons[1]
grid[1,0] = a_buttons[2]
grid[1,1] = a_buttons[3]

for p in range(12, 18):
    grid[1,p] = a_buttons[p-8]

grid[2,0] = a_buttons[10]
grid[2,1] = a_buttons[11]

for n in range(12,18):
    grid[2,n] = a_buttons[n]

for m in range(18):
    grid[3,m] = a_buttons[18+m]

for k in range(18):
    grid[4,k] = a_buttons[36+k]

grid[5,0] = a_buttons[54]
grid[5,1] = a_buttons[55]
# grid[5,2] = the Lanthanide series hole

for i in range(3, 18):
    grid[5,i] = a_buttons[68+i]

grid[6,0] = a_buttons[86]
grid[6,1] = a_buttons[87]
# grid[6,2] = the Actinide series hole

for j in range(2, 17):
    # Lanthanide series
    grid[7,j] = a_buttons[54+j]
#Actinide series
grid[8,2] = a_buttons[88]
grid[8,3] = a_buttons[89]

# The periodic table button functions
output = widgets.Output()
#display(output)


# @output.capture()


for i in range(len(a_buttons)):
    def on_clicked(b):
       selectAtom.value = i + 1
    a_buttons[i].on_click(on_clicked)


In [ ]:
""" Defining the gui widgets """
selectAtom = widgets.Dropdown(
    options= elements,
    value=2, 
    description='Atom:',
)
atomsLabel = widgets.Checkbox(
    value=False,
    description='Atomic label',
    disabled=False,
    indent=False
)
emission_type= widgets.Dropdown(
    options=['plane', 'cones'],
    value='plane', 
    description='Emission:',
)
includeNeutrons = widgets.Checkbox(
    value=True,
    description='Show neutrons',
    disabled=False,
    indent=False
)
colored_emissions = widgets.Checkbox(
    value=False,
    description='Color emissions',
    disabled=False,
    indent=False,
)
# Changes the equatorial(or cone) emission radius
emRSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 8.0,
    step= 0.1,
    description='EM_RADIUS',
)
# Changes the distance tween adjacent orthogonal protons (or protonstacks) 
s1s2Slider = widgets.FloatSlider(
    value = 2.5,
    min = 2.0,
    max = 3.0,
    step= 0.01,
    description='o-slots gap',
)
# Changes the pole to pole distance between adjacent e-p-n's
p2p3Slider = widgets.FloatSlider(     
    value = 3.0,
    min = 2.0,
    max = 4.0,
    step= 0.01,
    description='epn gap',
)
# Changes the neutron orbital radius
nOSlider = widgets.FloatSlider(
    value = 4.0,
    min = 2.0,
    max = 6.0,
    step= 0.1,
    description='N offset',
)
# Changes the cone emission angle
coneAngleSlider = widgets.FloatSlider(
    value = 30,  # deg  
    min = 20,   
    max = 40,  
    step= 0.1,
    description='Cone angle',
)
cameraType= widgets.Dropdown(
    options=['orthographic', 'perspective'],
    value='perspective', 
    description='View:',
)
cameraType2 = widgets.Dropdown(
    options=['horizontal', 'vertical'],
    value='vertical', 
    description='Z Axis:',
)

# The guiwidget control panel 
guiwidgets = widgets.VBox([selectAtom,atomsLabel,emission_type,
        colored_emissions,includeNeutrons,emRSlider,
        s1s2Slider,p2p3Slider,nOSlider,coneAngleSlider,cameraType,
        cameraType2]
)

# Observe any changes in widget values, button or gui
number = selectAtom.value
def handle_change(change):
    global number
    number = change.new
selectAtom.observe(handle_change, 'value')

display_neutrons = includeNeutrons.value
def handle_change(change):
    global display_neutrons 
    display_neutrons = change.new
includeNeutrons.observe(handle_change, 'value')

emR = emRSlider.value
def handle_change(change):
    global emR
    emR = change.new
emRSlider.observe(handle_change, 'value');

s1s2 = s1s2Slider.value  # Dist tween 2 ortho protons 
def handle_change(change):
    global s1s2
    s1s2 = change.new
s1s2Slider.observe(handle_change, 'value')

p2p3Dist = p2p3Slider.value # between adjacent e-p-n's
def handle_change(change):
    global p2p3Dist
    p2p3Dist = change.new
p2p3Slider.observe(handle_change, 'value')

neutron_orbit_radius = nOSlider.value # between adjacent e-p-n's
def handle_change(change):
    global neutron_orbit_radius
    neutron_orbit_radius = change.new
nOSlider.observe(handle_change, 'value')

c_angle_degree = coneAngleSlider.value # cone angle
def handle_change(change):
    global c_angle_degree
    c_angle_degree = change.new
coneAngleSlider.observe(handle_change, 'value')

cameraMode = cameraType.value
def handle_change(change):
    global cameraMode 
    cameraMode = change.new
cameraType.observe(handle_change, 'value')

cameraOrient = cameraType2.value
def handle_change(change):
    global cameraOrient 
    cameraOrient = change.new
cameraType2.observe(handle_change, 'value')



The Periodic Table. The periodic table below is provided as a sensible way to select the desired atom, just click on it. The many atomic properties the table is known to provide may be taken into consideration while making a choice.

Most of the controls will be displayed in the cell following the periodic table. Note that within those controls, the atom can also be selected by a dropdown widget.

Other control options include: type of proton emissions displayed, color coding of the specific number of protons in each proton stack, proton separation distances, and whether to include an atomic label or not.

In [ ]:
grid # Display the Periodic Table

In [ ]:
guiwidgets # Display the main control widgets. 

In [ ]:
# Atom data.

# Operating instructions. Start by clicking on this cell. 
# Next, from the Cell dropdown selection menu choose "Run 
# all Above". When the gui in the preceeding cell above 
# becomes visible, make any changes to it or select an atom 
# from the Periodic table cell. Next, with either the gui cell 
# or the Periodic table cell active, select "Run all Below"
# from the Cell dropdown menu. You'll then view the main 
# program outputs from the tab enclosure. Go back to the gui 
# or Periodic table to make any additional changes, then 
# select "Run all Below".  

# Atom data. After making an element or molecule change, 
# obtain the seleected set of atomic data.

# Selected Atom
An = elements[number-1]
atomicLabel = An[0] 
atomicNumber = number
elementlist = df['Element'].drop_duplicates().array
atom = elementlist[number-1]
atomsymbol = dlist[number-1]
rowstart = 19 * (atomicNumber - 1)
rowstop = rowstart + 18 # Used with the loc slicing
slotsetA_df = df.loc[rowstart:rowstop, 'SlotNumber':'AtomicSlotGrp'].set_index('SlotNumber')
slotlistA = list(slotsetA_df.loc[:, 'Protons'])
                    
#print('slotlistA = ',slotlistA)
#print(slotsetA_df) 
#slotsetA_df.info()  


In [ ]:
# Create the Slotlayout Diagram. 

# The slotlist, a set of 19 integers identifying 
# the number of protons in each numbered slot is
# found in the df['Protons'] column.

# Using 45 non-interactive buttons to create the
# atom's slotlayout (19 + 19 + 7 + 1) diagram: 19 
# in the center SL configuration; 19 for the left
# (1,3,5,...,19) and right(2,4,6,...,18) columns;
# 7 for the slot/proton count color legend at the
# bottom; and 1 atomic symbol/mumber at the bottom 
# right. All buttons except the bottom right incl:   
# proton count color, layout width and layout height.

# The 7 colors used for color-coding each slot's 
# total proton counts, 0-6.
sclrs = ['white','lightgray','dodgerblue','pink','salmon','greenyellow','skyblue']     

sdlist = slotlistA

# The colors for the 19 slots, sclist, according to
# the number of protons in each.
sclist = np.zeros(19, dtype='object')
for i in range(19):
    sclist[i] = sclrs[slotlistA[i]]

# Defining the slot buttons by adding the slot position, count and colors
letters = ['Z', 'Y', 'Y', 'Z', 'Z', 'Y', 'Y', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z']
s_buttons = []

for i in range(19):
    s_buttons.append(Button(description=f'{i+1} {letters[i]} {sdlist[i]}' , style=dict(button_color=sclist[i])))

for i in range(7):
    s_buttons.append(Button(description=str(i), style=dict(button_color=sclrs[i])))

for i in range(19):
    s_buttons.append(Button(description=f'{i+1} {letters[i]}', style=dict(button_color=sclist[i])))

# The slotlayout atom
s_buttons.append(Button(description=dlist[number-1]+' '+str(number), style=dict(button_color=sclrs[0],font_weight='bold')))

for button in s_buttons:
    button.layout = widgets.Layout(border = '1px solid black') 
    button.layout.width = '100%'
    button.layout.height = '100%' 

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
sgrid = GridspecLayout(10, 9)

sgrid[1,4] = s_buttons[13] # Cr6-Elements.csv SlotLayout.
sgrid[2,4] = s_buttons[3]
sgrid[2,5] = s_buttons[8]
sgrid[2,6] = s_buttons[12]
sgrid[3,3] = s_buttons[17]
sgrid[3,4] = s_buttons[1]
sgrid[3,5] = s_buttons[15]
sgrid[4,2] = s_buttons[9]
sgrid[4,3] = s_buttons[5]
sgrid[4,4] = s_buttons[0]
sgrid[4,5] = s_buttons[6]
sgrid[4,6] = s_buttons[11]
sgrid[5,3] = s_buttons[16]
sgrid[5,4] = s_buttons[2]
sgrid[5,5] = s_buttons[18]
sgrid[6,2] = s_buttons[10]
sgrid[6,3] = s_buttons[7]
sgrid[6,4] = s_buttons[4]
sgrid[7,4] = s_buttons[14]
sgrid[9,1] = s_buttons[19] # Color/num protons legend
sgrid[9,2] = s_buttons[20] # along the bottom row
sgrid[9,3] = s_buttons[21]
sgrid[9,4] = s_buttons[22]
sgrid[9,5] = s_buttons[23]
sgrid[9,6] = s_buttons[24]
sgrid[9,7] = s_buttons[25]
sgrid[0,0] = s_buttons[26] # L/R Vertical slot columns
sgrid[0,8] = s_buttons[27] 
sgrid[1,0] = s_buttons[28]
sgrid[1,8] = s_buttons[29]
sgrid[2,0] = s_buttons[30]
sgrid[2,8] = s_buttons[31]
sgrid[3,0] = s_buttons[32]
sgrid[3,8] = s_buttons[33]
sgrid[4,0] = s_buttons[34]
sgrid[4,8] = s_buttons[35]
sgrid[5,0] = s_buttons[36]
sgrid[5,8] = s_buttons[37]
sgrid[6,0] = s_buttons[38]
sgrid[6,8] = s_buttons[39]
sgrid[7,0] = s_buttons[40]
sgrid[7,8] = s_buttons[41]
sgrid[8,0] = s_buttons[42]
sgrid[8,8] = s_buttons[43]
sgrid[9,0] = s_buttons[44]
sgrid[9,8] = s_buttons[45] # Atomic symbpl and number


The Slot Layout Diagram.

al_l atoms can be described by the number of protons (0-6) in each of the atom's (1-19) occupied slot positions.

The main vertical up/down channel slots (top to bottom) are (14,4,2,1,3,5,15). There is also the front/back channel (11,8,1,9,13), and left/right channel (10,6,1,7,12); or refer to the directions from center slot 1: +/-x, +/-y, and main vertical column +/-z. The front/back/ left/right arms spin as a single group, the Carousel, around slot 1. Hydroden has a single proton in its single occupied slot 1. Helium's single slot 1 contains two protons. Hook slot positions: 16,17,18, and 19, are joined to the main up/down column at slots, 2 and 3.

al_l but most of the bottom row buttons contain number pairs separated by either X, Y or Z, the direction of that slot's proton equatorial emission plane viewed edgewise, orthogonal to the proton's spin axis. The left side number indicates the 1-19 slot number. The right side number indicates the number of protons in that atom's slot.

The Slot Layout is shown in the atomic configuration in the center of the diagram. It is a 3D, six pointed star projected onto the diagram's flat surface. The slot button information is repeated in the left and right side vertical columns for convienence. The seven center buttons in the bottom row contains the total protons per slot color legend. The selected atom’s symbol and atomic number are in the bottom right corner.

In [ ]:
""" Create an SL Diagram including slot proton spin directions: 
R, L, A or B; and proton pole electron positions: T, B, or A. An 
'Electrons' value of A means electrons are in a TB configuration, 
at the top proton pole positions at the positive end of slot, and 
at the bottom pole positions at the slot's negative end. BT 
electron positions are not allowed.
A 'SlotSpin' value of A means the 2-directional slot proton at 
the top spins left, while the bottom proton spins right. A value 
of B means the opposite, the proton the top spins right while 
the bottom proton spins left. 
Occupied slots in the center SL contain 5 digits: 1. Electron 
position at the proton's T or B or A (both T and B). 2. The slot 
number. 3. The slot's proton emission plane orientation. 4. The 
color-coded number of protons in the slot. 5. The slot's R,L,A 
(LR) or B (RL) proton spin directions. This atomic model 
simplifies things in that each proton may be accompanied by both 
an electron and a neutron which spin about the proton poles as a 
single object. Each object should spin independently"""

# The 7 colors used for color-coding each slot's total proton counts, 0-6.
snclrs = ['white','lightgray','dodgerblue','pink','salmon','greenyellow','skyblue']     

sndlist = slotlistA

# The colors for the 19 slots, sclist, according to
# the number of protons in each.
snclist = np.zeros(19, dtype='object')
for i in range(19):
    snclist[i] = snclrs[slotlistA[i]]

# Defining the slot buttons by adding the slot position, count and colors.
# If the slot is occupied, include 5 digits, unoccupied slots show 3 digits. 
if slotsetA_df.loc[1,'Protons'] == 0 :
    sn1 = Button(description = '1Z' + str(sndlist[0]), style=dict(button_color=snclist[0]))
elif slotsetA_df.loc[1,'Protons'] > 0 :
    sn1 = Button(description = str(slotsetA_df.loc[1,'Electrons']) + '1Z' + str(sndlist[0]) + str(slotsetA_df.loc[1,'SlotSpin']), style=dict(button_color=snclist[0]))
if slotsetA_df.loc[2,'Protons'] == 0 :
    sn2 = Button(description = '2Y' + str(sndlist[1]), style=dict(button_color=snclist[1]))
elif slotsetA_df.loc[2,'Protons'] > 0 :
    sn2 = Button(description = str(slotsetA_df.loc[2,'Electrons']) + '2Y' + str(sndlist[1]) + str(slotsetA_df.loc[2,'SlotSpin']), style=dict(button_color=snclist[1]))
if slotsetA_df.loc[3,'Protons'] == 0 :
    sn3 = Button(description = '3Y' + str(sndlist[2]), style=dict(button_color=snclist[2]))
elif slotsetA_df.loc[3,'Protons'] > 0 :
    sn3 = Button(description = str(slotsetA_df.loc[3,'Electrons']) + '3Y' + str(sndlist[2]) + str(slotsetA_df.loc[3,'SlotSpin']), style=dict(button_color=snclist[2]))
if slotsetA_df.loc[4,'Protons'] == 0 :
    sn4 = Button(description = '4Z' + str(sndlist[3]), style=dict(button_color=snclist[3]))
elif slotsetA_df.loc[4,'Protons'] > 0 :
    sn4 = Button(description = str(slotsetA_df.loc[4,'Electrons']) + '4Z' + str(sndlist[3]) + str(slotsetA_df.loc[4,'SlotSpin']), style=dict(button_color=snclist[3]))
if slotsetA_df.loc[5,'Protons'] == 0 :
    sn5 = Button(description = '5Z' + str(sndlist[4]), style=dict(button_color=snclist[4]))
elif slotsetA_df.loc[5,'Protons'] > 0 :
    sn5 = Button(description = str(slotsetA_df.loc[5,'Electrons']) + '5Z' + str(sndlist[4]) + str(slotsetA_df.loc[5,'SlotSpin']), style=dict(button_color=snclist[4]))
if slotsetA_df.loc[6,'Protons'] == 0 :
    sn6 = Button(description = '6Y' + str(sndlist[5]), style=dict(button_color=snclist[5]))
elif slotsetA_df.loc[6,'Protons'] > 0 :
    sn6 = Button(description = str(slotsetA_df.loc[6,'Electrons']) + '6Y' + str(sndlist[5]) + str(slotsetA_df.loc[6,'SlotSpin']), style=dict(button_color=snclist[5]))
if slotsetA_df.loc[7,'Protons'] == 0 :
    sn7 = Button(description = '7Y' + str(sndlist[6]), style=dict(button_color=snclist[6]))
elif slotsetA_df.loc[7,'Protons'] > 0 :
    sn7 = Button(description = str(slotsetA_df.loc[7,'Electrons']) + '7Y' + str(sndlist[6]) + str(slotsetA_df.loc[7,'SlotSpin']), style=dict(button_color=snclist[6]))
if slotsetA_df.loc[8,'Protons'] == 0 :
    sn8 = Button(description = '8X' + str(sndlist[7]), style=dict(button_color=snclist[7]))
elif slotsetA_df.loc[8,'Protons'] > 0 :
    sn8 = Button(description = str(slotsetA_df.loc[8,'Electrons']) + '8X' + str(sndlist[7]) + str(slotsetA_df.loc[8,'SlotSpin']), style=dict(button_color=snclist[7]))
if slotsetA_df.loc[9,'Protons'] == 0 :
    sn9 = Button(description = '9X' + str(sndlist[8]), style=dict(button_color=snclist[8]))
elif slotsetA_df.loc[9,'Protons'] > 0 :
    sn9 = Button(description = str(slotsetA_df.loc[9,'Electrons']) + '9X' + str(sndlist[8]) + str(slotsetA_df.loc[9,'SlotSpin']), style=dict(button_color=snclist[8]))
if slotsetA_df.loc[10,'Protons'] == 0 :
    sn10 = Button(description = 'AZ' + str(sndlist[9]), style=dict(button_color=snclist[9]))
elif slotsetA_df.loc[10,'Protons'] > 0 :
    sn10 = Button(description = str(slotsetA_df.loc[10,'Electrons']) + 'AZ' + str(sndlist[9]) + str(slotsetA_df.loc[10,'SlotSpin']), style=dict(button_color=snclist[9]))
if slotsetA_df.loc[11,'Protons'] == 0 :
    sn11 = Button(description = 'BZ' + str(sndlist[10]), style=dict(button_color=snclist[10]))
elif slotsetA_df.loc[11,'Protons'] > 0 :
    sn11 = Button(description = str(slotsetA_df.loc[11,'Electrons']) + 'BZ' + str(sndlist[10]) + str(slotsetA_df.loc[11,'SlotSpin']), style=dict(button_color=snclist[10]))
if slotsetA_df.loc[12,'Protons'] == 0 :
    sn12 = Button(description = 'CZ' + str(sndlist[11]), style=dict(button_color=snclist[11]))
elif slotsetA_df.loc[12,'Protons'] > 0 :
    sn12 = Button(description = str(slotsetA_df.loc[12,'Electrons']) + 'CZ' + str(sndlist[11]) + str(slotsetA_df.loc[12,'SlotSpin']), style=dict(button_color=snclist[11]))
if slotsetA_df.loc[13,'Protons'] == 0 :
    sn13 = Button(description = 'DZ' + str(sndlist[12]), style=dict(button_color=snclist[12]))
elif slotsetA_df.loc[13,'Protons'] > 0 :
    sn13 = Button(description = str(slotsetA_df.loc[13,'Electrons']) + 'DZ' + str(sndlist[12]) + str(slotsetA_df.loc[13,'SlotSpin']), style=dict(button_color=snclist[12]))
if slotsetA_df.loc[14,'Protons'] == 0 :
    sn14 = Button(description = 'EY' + str(sndlist[13]), style=dict(button_color=snclist[13]))
elif slotsetA_df.loc[14,'Protons'] > 0 :
    sn14 = Button(description = str(slotsetA_df.loc[14,'Electrons']) + 'EY' + str(sndlist[13]) + str(slotsetA_df.loc[14,'SlotSpin']), style=dict(button_color=snclist[13]))
if slotsetA_df.loc[15,'Protons'] == 0 :
    sn15 = Button(description = 'FY' + str(sndlist[14]), style=dict(button_color=snclist[14]))
elif slotsetA_df.loc[15,'Protons'] > 0 :
    sn15 = Button(description = str(slotsetA_df.loc[15,'Electrons']) + 'FY' + str(sndlist[14]) + str(slotsetA_df.loc[15,'SlotSpin']), style=dict(button_color=snclist[14]))
if slotsetA_df.loc[16,'Protons'] == 0 :
    sn16 = Button(description = 'GZ' + str(sndlist[15]), style=dict(button_color=snclist[15]))
elif slotsetA_df.loc[16,'Protons'] > 0 :
    sn16 = Button(description = str(slotsetA_df.loc[16,'Electrons']) + 'GZ' + str(sndlist[15]) + str(slotsetA_df.loc[16,'SlotSpin']), style=dict(button_color=snclist[15]))
if slotsetA_df.loc[17,'Protons'] == 0 :
    sn17 = Button(description = 'HZ' + str(sndlist[16]), style=dict(button_color=snclist[16]))
elif slotsetA_df.loc[17,'Protons'] > 0 :
    sn17 = Button(description = str(slotsetA_df.loc[17,'Electrons']) + 'HZ' + str(sndlist[16]) + str(slotsetA_df.loc[17,'SlotSpin']), style=dict(button_color=snclist[16]))
if slotsetA_df.loc[18,'Protons'] == 0 :
    sn18 = Button(description = 'IZ' + str(sndlist[17]), style=dict(button_color=snclist[17]))
elif slotsetA_df.loc[18,'Protons'] > 0 :
    sn18 = Button(description = str(slotsetA_df.loc[18,'Electrons']) + 'IZ' + str(sndlist[17]) + str(slotsetA_df.loc[18,'SlotSpin']), style=dict(button_color=snclist[17]))
if slotsetA_df.loc[19,'Protons'] == 0 :
    sn19 = Button(description = 'JZ' + str(sndlist[18]), style=dict(button_color=snclist[18]))
elif slotsetA_df.loc[19,'Protons'] > 0 :
    sn19 = Button(description = str(slotsetA_df.loc[19,'Electrons']) + 'JZ' + str(sndlist[18]) + str(slotsetA_df.loc[19,'SlotSpin']), style=dict(button_color=snclist[18]))

# Slotlayout proton total color legend 
sn20 = Button(description=str('0'), style=dict(button_color=snclrs[0]))
sn21 = Button(description=str('1'), style=dict(button_color=snclrs[1]))
sn22 = Button(description=str('2'), style=dict(button_color=snclrs[2]))
sn23 = Button(description=str('3'), style=dict(button_color=snclrs[3]))
sn24 = Button(description=str('4'), style=dict(button_color=snclrs[4]))
sn25 = Button(description=str('5'), style=dict(button_color=snclrs[5]))
sn26 = Button(description=str('6'), style=dict(button_color=snclrs[6]))
# Left and right slot columns, 
#sn27 = Button(description='1 Z '+str(sndlist[0]), style=dict(button_color=snclist[0]))
sn27 = Button(description = '1Z' + str(sndlist[0]), style=dict(button_color=snclist[0]))
sn28 = Button(description = '2Y' + str(sndlist[1]), style=dict(button_color=snclist[1]))
sn29 = Button(description = '3Y' + str(sndlist[2]), style=dict(button_color=snclist[2]))
sn30 = Button(description = '4Z' + str(sndlist[3]), style=dict(button_color=snclist[3]))
sn31 = Button(description = '5Z' + str(sndlist[4]), style=dict(button_color=snclist[4]))
sn32 = Button(description = '6Y' + str(sndlist[5]), style=dict(button_color=snclist[5]))
sn33 = Button(description = '7Y' + str(sndlist[6]), style=dict(button_color=snclist[6]))
sn34 = Button(description = '8X' + str(sndlist[7]), style=dict(button_color=snclist[7]))
sn35 = Button(description = '9X' + str(sndlist[8]), style=dict(button_color=snclist[8]))
sn36 = Button(description = 'AZ' + str(sndlist[9]), style=dict(button_color=snclist[9]))
sn37 = Button(description = 'BZ' + str(sndlist[10]), style=dict(button_color=snclist[10]))
sn38 = Button(description = 'CZ' + str(sndlist[11]), style=dict(button_color=snclist[11]))
sn39 = Button(description = 'DZ' + str(sndlist[12]), style=dict(button_color=snclist[12]))
sn40 = Button(description = 'EY' + str(sndlist[13]), style=dict(button_color=snclist[13]))
sn41 = Button(description = 'FY' + str(sndlist[14]), style=dict(button_color=snclist[14]))
sn42 = Button(description = 'GZ' + str(sndlist[15]), style=dict(button_color=snclist[15]))
sn43 = Button(description = 'HZ' + str(sndlist[16]), style=dict(button_color=snclist[16]))
sn44 = Button(description = 'IZ' + str(sndlist[17]), style=dict(button_color=snclist[17]))
sn45 = Button(description = 'JZ' + str(sndlist[18]), style=dict(button_color=snclist[18]))
# The slotlayout atom
sn46 = Button(description=dlist[number-1]+' '+str(number), style=dict(button_color=sclrs[0],font_weight='bold'))

# Creating the list of buttons: 19,7,19 and 1.
snbttns = [sn1,sn2,sn3,sn4,sn5,sn6,sn7,sn8,sn9,sn10,sn11,sn12,sn13,sn14,sn15,sn16,sn17,sn18,sn19,
          sn20,sn21,sn22,sn23,sn24,sn25,sn26,sn27,sn28,sn29,sn30,sn31,sn32,sn33,sn34,sn35,sn36,
          sn37,sn38,sn39,sn40,sn41,sn42,sn43,sn44,sn45,sn46]

# Add each button's border, layout width and height.
i=0
while i < 46 :
    snbttns[i].layout = widgets.Layout(border = '1px solid black') 
    snbttns[i].layout.width = '100%'
    snbttns[i].layout.height = '100%' 
    i += 1

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
sngrid = GridspecLayout(10, 9)

sngrid[1,4] = snbttns[13] # Cr6-Elements.csv SlotLayout.
sngrid[2,4] = snbttns[3]
sngrid[2,5] = snbttns[8]
sngrid[2,6] = snbttns[12]
sngrid[3,3] = snbttns[17]
sngrid[3,4] = snbttns[1]
sngrid[3,5] = snbttns[15]
sngrid[4,2] = snbttns[9]
sngrid[4,3] = snbttns[5]
sngrid[4,4] = snbttns[0]
sngrid[4,5] = snbttns[6]
sngrid[4,6] = snbttns[11]
sngrid[5,3] = snbttns[16]
sngrid[5,4] = snbttns[2]
sngrid[5,5] = snbttns[18]
sngrid[6,2] = snbttns[10]
sngrid[6,3] = snbttns[7]
sngrid[6,4] = snbttns[4]
sngrid[7,4] = snbttns[14]
sngrid[9,1] = snbttns[19] # Color/num protons legend
sngrid[9,2] = snbttns[20] # along the bottom row
sngrid[9,3] = snbttns[21]
sngrid[9,4] = snbttns[22]
sngrid[9,5] = snbttns[23]
sngrid[9,6] = snbttns[24]
sngrid[9,7] = snbttns[25]
sngrid[0,0] = snbttns[26] # L/R Vertical slot columns
sngrid[0,8] = snbttns[27] 
sngrid[1,0] = snbttns[28]
sngrid[1,8] = snbttns[29]
sngrid[2,0] = snbttns[30]
sngrid[2,8] = snbttns[31]
sngrid[3,0] = snbttns[32]
sngrid[3,8] = snbttns[33]
sngrid[4,0] = snbttns[34]
sngrid[4,8] = snbttns[35]
sngrid[5,0] = snbttns[36]
sngrid[5,8] = snbttns[37]
sngrid[6,0] = snbttns[38]
sngrid[6,8] = snbttns[39]
sngrid[7,0] = snbttns[40]
sngrid[7,8] = snbttns[41]
sngrid[8,0] = snbttns[42]
sngrid[8,8] = snbttns[43]
sngrid[9,0] = snbttns[44]
sngrid[9,8] = snbttns[45] # Atomic symbol and number


In [ ]:
# Build each atom from Cr6-Elements

# atomicNumber = number  
# Select the desired atom from the dropdown widget or table.
if 1 <= number < 13:
    CAROUSEL_ROTATION_SECONDS = 2.5
if 14 <= number < 20:
    CAROUSEL_ROTATION_SECONDS = 4.0
if 21 <= number :
    CAROUSEL_ROTATION_SECONDS = 6.0

grpA1 = Group()  # Carousel. The corner rotation widget spins grpA1, 
# slots 6,7,8,9,10,11,12,13.
grpA2 = Group()  # grpA2 contains slot1 and the remaining non-carousel
# orthogonal top, bottom and hook slots.
grpB1 = Group()  # Atom 2 Slot 1 and carousel
grpB2 = Group()  # Atom 2 remaining slots

# Each of the 1-19 slots of a given atom may have either right 
# (A stacks) or left (B stacks) spinning proton groups. 38 Total.

stacks_A = [Group() for _ in range(19)]

stacks_B = [Group() for _ in range(19)]

""" 
Widget Controlled variables 1. global emR. EM_RADIUS. The equatorial 
plane (or cone) emission radius displayed. 2. global s1s2. o-slots 
gap, the distance between orthogonal protons or orthogonal proton 
stacks. 3. Global p2p3Dist. epn gap. The distance between adjacent 
e-p-n proton groups. 4. global neutron_orbit_radius. The orbital offfset 
distance about the direct charge channel. 5. global c_angle_degree, the 
proton emission cone angle in degrees.
"""
# Dimensions # PROTON_RADIUS is now in cell In[1]
NEUTRON_RADIUS = PROTON_RADIUS*1.125
ELECTRON_RADIUS = 2*PROTON_RADIUS/7.5
EM_RADIUS = 2*emR*PROTON_RADIUS # 2*pE - PROTON_RADIUS/2
CONE_ANGLE = c_angle_degree*2*PI_PHI/360
CONE_HEIGHT = EM_RADIUS*tan(CONE_ANGLE)
p12 = 2*s1s2*PROTON_RADIUS
pE = p12
p2p3 = 2*p2p3Dist*PROTON_RADIUS
p1e1 = PROTON_RADIUS + 0.05*p12 # proton-electron separation
n1n2 = 2*NEUTRON_RADIUS*neutron_orbit_radius

# Proton set Geometries and Materials # Color.js Find a nice color spectrum
proton_geometry =  SphereBufferGeometry(PROTON_RADIUS, 24, 16)
neutron_geometry =  SphereBufferGeometry(NEUTRON_RADIUS, 24, 16)
electron_geometry =  SphereBufferGeometry(ELECTRON_RADIUS, 8, 16)
emission_geometry_P = CircleGeometry(EM_RADIUS, 24)
emission_geometry_X = ConeGeometry( EM_RADIUS, CONE_HEIGHT, 16, openEnded = True )
proton_material = MeshStandardMaterial(color='blue')
neutron_material = MeshStandardMaterial(color='red')
electron_material = MeshStandardMaterial(color='cyan') #'orangered','salmon','green','cyan','teal','orange'
emission_material7 = MeshStandardMaterial(color='gray',side='DoubleSide',transparent = True,opacity = 0.20)
emission_material1 = MeshStandardMaterial(color='magenta',side='DoubleSide',transparent = True,opacity = 0.20)
emission_material2 = MeshStandardMaterial(color='blue',side='DoubleSide',transparent = True,opacity = 0.20)
emission_material3 = MeshStandardMaterial(color='pink',side='DoubleSide',transparent = True,opacity = 0.20)
emission_material4 = MeshStandardMaterial(color='salmon',side='DoubleSide',transparent = True,opacity = 0.20)
emission_material5 = MeshStandardMaterial(color='lime',side='DoubleSide',transparent = True,opacity = 0.20)
emission_material6 = MeshStandardMaterial(color='skyblue',side='DoubleSide',transparent = True,opacity = 0.20)

emission_material_list = [emission_material1,emission_material2,emission_material3,
        emission_material4,emission_material5,emission_material6,emission_material7]

if emission_type.value == 'plane':
    emission_geometry = emission_geometry_P
if emission_type.value == 'cones':
    emission_geometry = emission_geometry_X

def buildProStackM(a,j): 
    """
    Parameters: a - total number of slot protons. j - given slot #.
    
    buildProStackM(a,j) is an attempt to follow Miles Mathis' 
    paper,'Diatomic Hydrogen' (DH), which includes electron-proton 
    (e-p) to e-p charge bonding summarized in rules 1, 2, and 3: 
    Rule 1. Two protons, each with any spin direction (L or R) can 
    bond when they share a T B electron position configuration.
    Rule 2. T T or B B electron position configurations will be valid 
    only if both protons have the same (L or R) spin direction.
    Rule 3. No combination of B T electrons are valid, two electrons 
    between two protons prevent bonding.
    """
    
    if colored_emissions.value == True:
        for k in range(6): 
            if a == k + 1 :
                emission_material = emission_material_list[k]
    else:
        emission_material = emission_material_list[6]              

    """ 
    #p2p3 = 2*p2p3Dist*PROTON_RADIUS, distance added for each additional e-p.
    The six sets of equidistant proton positions for a slot with 1-6 protons. 
    The proton (groups) are added beginning at the slot center. 5,3,1,2,4,6.
    """
    
    match a:
        case 1:
            proton_positions = [0]
        case 2:
            proton_positions = [p2p3/2,-p2p3/2]
        case 3:
            proton_positions = [0,-p2p3, p2p3]
        case 4:
            proton_positions = [p2p3/2,-p2p3/2,3*p2p3/2,-3*p2p3/2]
        case 5:
            proton_positions = [0,-p2p3,p2p3,-2*p2p3,2*p2p3]        
        case 6:
            proton_positions = [p2p3/2,-p2p3/2,3*p2p3/2,-3*p2p3/2,5*p2p3/2,-5*p2p3/2]  
    
    def cones_plus_plane(proton_mi,emission1,emission2,position):
        emission1.add(Mesh(emission_geometry, emission_material))
        #emission1.position = [0,0,CONE_HEIGHT/2]
        emission1.position = [0,0,position]
        emission1.rotateX(-PI_PHI/2)
        proton_mi.add(emission1)
        emission2.add(Mesh(emission_geometry, emission_material))
        #emission2.position = [0,0,-CONE_HEIGHT/2]
        emission2.position = [0,0,-position]
        emission2.rotateX(PI_PHI/2)  
        proton_mi.add(emission2)            
        proton_mi.add(Mesh(emission_geometry_P, emission_material))       
        return

    def add_electron(proton_mi,electron_mi,position):
        electron_mi.add(Mesh(electron_geometry, electron_material))
        electron_mi.position = [0,-n1n2/10,position]
        electron_mi.rotateZ(random()*2*PI_PHI)
        proton_mi.add(electron_mi)    
        return
    
    def single_neutron(proton_mi,neutron,position): 
        neutron.add(Mesh(neutron_geometry, neutron_material))
        neutron.position = [0,n1n2/2,position]  
        neutron.rotateZ(random()*2*PI_PHI)
        if display_neutrons is True :
            proton_mi.add(neutron) 
        return  

    def pair_of_neutrons(proton_mi,neutron1,neutron2,position):
        neutron1.add(Mesh(neutron_geometry, neutron_material))
        neutron1.position = [0,-n1n2/2,0]
        neutron2.add(Mesh(neutron_geometry, neutron_material))
        neutron2.position = [0,n1n2/2,0]
        neutron_pair.add(neutron1)
        neutron_pair.add(neutron2)
        neutron_pair.rotateZ(random()*2*PI_PHI)
        neutron_pair.position = [0,0,position]
        if display_neutrons is True :
            proton_mi.add(neutron_pair)        
        return

    num = 0 # Increments the occupied slot's 1-6 proton groups.  
    while num < a:
        proton_mi = Group()   
        proton_mi.add(Mesh(proton_geometry, proton_material))
        if emission_type.value == 'plane':
            proton_mi.add(Mesh(emission_geometry, emission_material))
        if emission_type.value == 'cones': 
            emission1 = Object3D() 
            emission2 = Object3D()
            cones_plus_plane(proton_mi,emission1,emission2,CONE_HEIGHT/2)

        # Electron and neutron additions to proton spin groups. 
        electron_mi = Object3D()        
        neutron_mi = Object3D()
        neutronNi = Object3D()
        neutron_pair = Group()

        # Building the various T,B,A, and R,L,A slot configurations. 
        intColumn = 8+num  # Pointing to the N1-N6 data locations. 
        intValue = slotsetA_df.iloc[j,intColumn]
        if slotsetA_df.loc[j+1,('SlotSpin')] == "R" or "L" : 
            if slotsetA_df.loc[j+1,('Electrons')] == "T" : 
                add_electron(proton_mi,electron_mi,p1e1)             
                if intValue == 1 :                  
                    single_neutron(proton_mi,neutron_mi,-p2p3/2)               
                elif intValue == 2 :  
                    pair_of_neutrons(proton_mi,neutron_mi,neutronNi,-p2p3/2)
            if slotsetA_df.loc[j+1,('Electrons')] == "B" : 
                add_electron(proton_mi,electron_mi,-p1e1)
                if intValue == 1 :  
                    single_neutron(proton_mi,neutron_mi,p2p3/2)
                elif intValue == 2 :
                    pair_of_neutrons(proton_mi,neutron_mi,neutronNi,p2p3/2)
            if slotsetA_df.loc[j+1,('Electrons')] == "A" : # 'SlotSpin' = R or L
                if num%2 != 0:
                    add_electron(proton_mi,electron_mi,-p1e1)
                    if intValue == 1 :                     
                        single_neutron(proton_mi,neutron_mi,p2p3/2)                    
                    elif intValue == 2 :
                        pair_of_neutrons(proton_mi,neutron_mi,neutronNi,p2p3/2)                    
                elif num%2 == 0:
                    add_electron(proton_mi,electron_mi,p1e1)
                    if intValue == 1 :                     
                        single_neutron(proton_mi,neutron_mi,-p2p3/2)                    
                    elif intValue == 2 :
                        pair_of_neutrons(proton_mi,neutron_mi,neutronNi,-p2p3/2)  
        if slotsetA_df.loc[j+1,('SlotSpin')] == "A" and slotsetA_df.loc[j+1,('Electrons')] == "A":
            if num == 0 :
                add_electron(proton_mi,electron_mi,p1e1)
                if intValue == 1 :                  
                    single_neutron(proton_mi,neutron_mi,p2p3/2)                
            if num == 1 :
                if intValue == 0 :   
                    add_electron(proton_mi,electron_mi,-p1e1)
                if intValue == 1 :
                    add_electron(proton_mi,electron_mi,p1e1)
                    single_neutron(proton_mi,neutron_mi,-p2p3/2)   
                if intValue == 2 :
                    add_electron(proton_mi,electron_mi,p1e1)
                    pair_of_neutrons(proton_mi,neutron_mi,neutronNi,-p2p3/2)                  
                    proton_mi.rotateX(PI_PHI)
                    proton_mi.rotateZ(random()*2*PI_PHI)
            if num > 1 and num%2 == 0:
                add_electron(proton_mi,electron_mi,p1e1)                
                if intValue == 1 :                     
                    single_neutron(proton_mi,neutron_mi,-p2p3/2)
                elif intValue == 2 :
                    pair_of_neutrons(proton_mi,neutron_mi,neutronNi,-p2p3/2)
            if num > 1 and num%2 != 0:          
                if intValue == 1 :                     
                    add_electron(proton_mi,electron_mi,-p1e1)
                    single_neutron(proton_mi,neutron_mi,p2p3/2)
                elif intValue == 2 :
                    add_electron(proton_mi,electron_mi,-p1e1)
                    pair_of_neutrons(proton_mi,neutron_mi,neutronNi,p2p3/2)
      
        proton_mi.rotateZ(random()*2*PI_PHI)
        proton_mi.position = [0,0,proton_positions[num]]
        
        # Slot spin assignments
        if slotsetA_df.loc[j+1,('SlotSpin')] == "R":                 
            stacks_A[j].add(proton_mi)
        elif slotsetA_df.loc[j+1,('SlotSpin')] == "L" :
            stacks_B[j].add(proton_mi)
        elif slotsetA_df.loc[j+1,('SlotSpin')] == "A" :        
            # Double spin slot spin assignments.
            if num%2 != 0:            
                stacks_A[j].add(proton_mi)
            else:
                stacks_B[j].add(proton_mi)  
        elif slotsetA_df.loc[j+1,('SlotSpin')] == "B" :        
            # Double spin slot spin assignments.
            if num%2 != 0:            
                stacks_B[j].add(proton_mi)
            else:
                stacks_A[j].add(proton_mi)               
               
        num += 1
    return

def Orient(n, stack):
    # proStack emission planes are initial_ly oriented in the z-plane.
    # Reorient the stack otherwise according to the "SlotOrien" column.
    if slotsetA_df.loc[n+1,('SlotOrien')] == "Y" : 
        stack.rotateX(-PI_PHI/2)            
    if slotsetA_df.loc[n+1,('SlotOrien')] == "X" :
        stack.rotateX(-PI_PHI/2)
        stack.rotateY(PI_PHI/2)
    return            

def Position(n,stack):
    # The default values are just used as position markers, +/- x, y or z 
    # with respect to (0,0,0). Use those directions but calculate the slot's 
    # position magnitude based on the values of p2p3, p12 and pE. 
    loc = (abs(slotsetA_df.loc[n+1,('P2P3')]*p2p3 
               + slotsetA_df.loc[n+1,('P12')]*p12 
               + slotsetA_df.loc[n+1,('PE')]*pE))
    # Determine the direction of the stack according to the presence and sign  
    # (+/-) of slotsetA_df's occupied (x,y,z) positions. Either a value or a
    #  zero, '0'. If there's a value, let it be given the position magnitude.
    px = slotsetA_df.loc[i+1,('PX')] #
    py = slotsetA_df.loc[i+1,('PY')] 
    pz = slotsetA_df.loc[i+1,('PZ')] 
    if px != 0 : px = copysign(1, px) 
    if py != 0 : py = copysign(1, py) 
    if pz != 0 : pz = copysign(1, pz)             
    stack.position = [px*loc, py*loc, pz*loc]
    return

# Assemble the atomic models.
for i in range( 19 ):
    if slotsetA_df.loc[i+1,('Protons')] > 0 :   
        numPs = slotsetA_df.loc[i+1,('Protons')]
        buildProStackM(numPs,i)     
        Orient(i,stacks_A[i])
        Orient(i,stacks_B[i])
        Position(i,stacks_A[i])
        Position(i,stacks_B[i])
        #grpA1: Carousal slots: 6,7,8,9,10,11,12,13
        anSlots=[5,6,7,8,9,10,11,12] # index#=slot-1        
        if i in anSlots:       
            grpA1.add(stacks_A[i])
            grpA1.add(stacks_B[i])
        else:
            grpA2.add(stacks_A[i]) 
            grpA2.add(stacks_B[i])   

print(An) # (' 2 helium He ', 2)
print(slotlistA) 
#print('moleculeOrNot ,',moleculeOrNot.value,', di-atom ,',numberM) 
#slotsetA_df.to_csv('out.csv') 

#slotsetA_df  

In [ ]:
# The rendered scene

if cameraOrient == 'horizontal':
    camera = camera_z_axis_horiz
else:
    camera = camera_z_axis_vert

if atomsLabel.value == True:   # Still shows UserWarning
    scene = Scene(children=[grpA1, grpA2, grpB1, grpB2, camera, key_light, ambient_light,
        make_text(atomicLabel,
                  height=0.25,
                  position = [-1,1,number*0.025 + 1.5])])
else:
    scene = Scene(children=[grpA1, grpA2, grpB1, grpB2, camera, key_light, ambient_light])

controller = OrbitControls(controlling=camera, screenSpacePanning=True)
camera.mode = cameraType.value
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)],
                    width=VIEW_WIDTH, height=VIEW_HEIGHT)


# initialising the scene
scene = Scene()

# Note. The UserWarning appears to be due to non JSON compliant infinite 
# values used in the pythreejs orbital camera. See the related issue. JSON 
# serialization of OrbitControls fails with jupyter_client 7.0.3,opened 
# on 22 Sep 21, https://github.com/jupyter-widgets/pythreejs/issues/366

# 16 April, switched to a combined camera, allowing the user to switch views 
# https://github.com/jupyter-widgets/pythreejs/blob/master/examples/Combined%20Camera.ipynb
# Still getting a User Warning. 

In [ ]:
# Atom 1 spin animations. Defining the rotation widget controls for slots 1 - 19. 
# Slot 1 (and the carousel).
a_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, CAROUSEL_ROTATION_SECONDS], values=[0, 2*PI_PHI])
# Slots 2-19.
b_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 2*PI_PHI])

# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.
# Defining the rotation widget controls for slots 1 - 19. 

# The remaining widgets rotate the individual slots. 
track_types = ['B' for _ in range(20)]
track_types[19] = 'A'
spin_clips = [AnimationClip(tracks=[a_track]) for _ in range(19)]
spin_actions = [AnimationAction(AnimationMixer(stacks_A[i]), spin_clips[i], stacks_A[i]) for i in range(19)]

spinA20_track = a_track
spinA20_clip = AnimationClip(tracks=[spinA20_track])
spinA20_action = AnimationAction(AnimationMixer(grpA1), spinA20_clip, grpA1)
spin_actions.append(spinA20_action)

rgridA = GridspecLayout(8, 7)

rgridA[1, 3] = spin_actions[14]
rgridA[2, 3] = spin_actions[4]
rgridA[2, 4] = spin_actions[9]
rgridA[2, 5] = spin_actions[13]
rgridA[3, 2] = spin_actions[18]
rgridA[3, 3] = spin_actions[2]
rgridA[3, 4] = spin_actions[16]
rgridA[4, 1] = spin_actions[10]
rgridA[4, 2] = spin_actions[6]
rgridA[4, 3] = spin_actions[1]
rgridA[4, 4] = spin_actions[7]
rgridA[4, 5] = spin_actions[12]
rgridA[5, 2] = spin_actions[17]
rgridA[5, 3] = spin_actions[3]
rgridA[5, 4] = spin_actions[19]
rgridA[6, 1] = spin_actions[11]
rgridA[6, 2] = spin_actions[8]
rgridA[6, 3] = spin_actions[5]
rgridA[7, 3] = spin_actions[15]
rgridA[7, 5] = spinA20_action

In [ ]:
# Second set of spin animations. Defining the rotation widget controls for slots 1 - 19. 
# Slot 1 (and the carousel).
a_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, CAROUSEL_ROTATION_SECONDS], values=[0, -2*PI_PHI])
# Slots 2-19.
b_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, -2*PI_PHI])

# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.
# Defining the rotation widget controls for slots 1 - 19. 
# The remaining widgets rotate the individual slots. 

track_types = ['B' for _ in range(20)]
track_types[19] = 'A'
spin_clips = [AnimationClip(tracks=[b_track]) for _ in range(19)]
spin_actions = [AnimationAction(AnimationMixer(stacks_B[i]), spin_clips[i], stacks_B[i]) for i in range(19)]

spinB20_clip = AnimationClip(tracks=[a_track])
spinB20_action = AnimationAction(AnimationMixer(grpB1), spinB20_clip, grpB1)
spin_actions.append(spinB20_action)

# Layout the 19 rotation controls
rgridB = GridspecLayout(8, 7)

rgridB[1, 3] = spin_actions[14]
rgridB[2, 3] = spin_actions[4]
rgridB[2, 4] = spin_actions[9]
rgridB[2, 5] = spin_actions[13]
rgridB[3, 2] = spin_actions[18]
rgridB[3, 3] = spin_actions[2]
rgridB[3, 4] = spin_actions[16]
rgridB[4, 1] = spin_actions[10]
rgridB[4, 2] = spin_actions[6]
rgridB[4, 3] = spin_actions[1]
rgridB[4, 4] = spin_actions[7]
rgridB[4, 5] = spin_actions[12]
rgridB[5, 2] = spin_actions[17]
rgridB[5, 3] = spin_actions[3]
rgridB[5, 4] = spin_actions[19]
rgridB[6, 1] = spin_actions[11]
rgridB[6, 2] = spin_actions[8]
rgridB[6, 3] = spin_actions[5]
rgridB[7, 3] = spin_actions[15]
#rgridB[7, 5] = spinB20_action

In [ ]:
# The widget tab enclosure.

# Including a non-interactive periodic table showing the atom selected.

b_buttons = []
for i in range(90):
    button = Button(description=dlist[i], style=dict(button_color=clist[i]))
    if i == atomicNumber-1 :
        button = Button(description=dlist[atomicNumber-1], style=dict(button_color='white',font_weight='bold'))
        button.layout = widgets.Layout(border = '2px solid red')
    else :
        button.layout = widgets.Layout(border = '1px solid black')
    button.layout.width = '100%'
    button.layout.height = '100%'  #'40px'
    b_buttons.append(button)

# Layout the 90 buttons in periodic table form
gridb = GridspecLayout(10, 18)

gridb[0,0] = b_buttons[0]
gridb[0,17] = b_buttons[1]
gridb[1,0] = b_buttons[2]
gridb[1,1] = b_buttons[3]
p=12
while p < 18:
    gridb[1,p] = b_buttons[p-8]
    p += 1
gridb[2,0] = b_buttons[10]
gridb[2,1] = b_buttons[11]
n=12
while n < 18:
    gridb[2,n] = b_buttons[n]
    n += 1
m=0
while m < 18:
    gridb[3,m] = b_buttons[18+m]
    m += 1
k=0
while k < 18:
    gridb[4,k] = b_buttons[36+k]
    k += 1
gridb[5,0] = b_buttons[54]
gridb[5,1] = b_buttons[55]
# gridb[5,2] = the Lanthanide series hole
i=3
while i < 18:
    gridb[5,i] = b_buttons[68+i]
    i += 1
gridb[6,0] = b_buttons[86]
gridb[6,1] = b_buttons[87]
# gridb[6,2] = the Actinide series hole
j=2
while j < 17:
    # Lanthanide series
    gridb[7,j] = b_buttons[54+j]
    j += 1
#Actinide series
gridb[8,2] = b_buttons[88]
gridb[8,3] = b_buttons[89]

# The tab enclosure 
tab1 = VBox(children=[renderer]) # 3D Atom
tab2 = VBox(children=[sgrid])  # Slotlayout (SL) diagram
tab3 = VBox(children=[sngrid]) # SL w #'s of' left/right neutrons
tab4 = VBox(children=[gridb])  # Periodic table

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, atomicLabel)
tab.set_title(1, dlist[number-1]+' Slotlayout (SL)')
tab.set_title(2, dlist[number-1]+' SL 1.TBA 5.RLAB')
tab.set_title(3, 'Periodic Table')

#VBox(children=[tab]) # Display a tab enclosure with all outputs.
# Displayed between the top (R) and bottom (L) rotation controls.

In [ ]:
"""Rotation controls for all slots' right spinning proton groups.
The Carousel's R spin control is at the bottom right. 
Less than half the slots conbtain only right spin groups and about as many 
other slots contain left spin groups. Some slots, i.e. slot1, contain both"""
rgridA  

In [ ]:
VBox(children=[tab]) # Display a tab enclosure with all outputs

In [ ]:
rgridB  # Rotation controls for just the left spinning proton groups in all slots.


Charge Field physics and the Unified Field theory, (and much more) have been well developed and well described by Miles Mathis. See,

***THE GREATEST STANDING ERRORS IN PHYSICS AND MATHEMATICS***
<http://milesmathis.com/index.html>
[Miles Mathis science site Homepage](http://milesmathis.com/index.html).

**SECTION 9: THE NUCLEUS** contains descriptions and diagrams 
of charge channeling and charge recycling by the 
elements. A paper most relevant to Atom Builder is 
>**How to Build the Elements**. 
>Explaining the periodic table, with nuclear diagrams".  
<http://milesmathis.com/nuclear.pdf>
[How to Build the Elements](http://milesmathis.com/nuclear.pdf)
For Charge Field discussion visit the forum at "Miles Mathis' Charge Field - Portal" https://milesmathis.forumotion.com.

This particular project is described in the Miles Periodic Table with Standard Periodic Table reference thread, https://milesmathis.forumotion.com/t634p75-miles-periodic-table-with-standard-periodic-table-reference#6702.

This project was Cr6's idea. He's also responsible for following Miles Mathis' atomic model and creating the Slotlayout diagram this project greatly relies.

There's at least one 10 year old charge field "Atom Viewer" out there, Nevyn's object oriented, way sophisticated javaScript code. I'll try to figure it out.

Please pardon my understandings, personal interpretations and mistakes. Feel free to make this project better.